# Update the survey metadata tables in the database

This code is to be run after step 02, which generates FlatRecordSpec and FlatValuesSpec csv files.

This code loads those files to the corresponding metadata tables in the database (currently `dhs_survey_specs.dhs_table_specs_flat` and `dhs_survey_specs.dhs_value_descs`). 

It checks the version of the files present as parsed CSVs and compares to what's present in the database in order to make sure that the database contains the latest info.

In [1]:
with open('config/pg_conn.txt') as conn_details:
    conn_str_psyco = conn_details.readline()
    conn_str_sqlalchemy = conn_details.readline()

In [2]:
import sys
sys.executable

'/home/harry/anaconda3/envs/geodev_38/bin/python'

In [3]:
import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg
from sqlalchemy import create_engine
import sqlalchemy as sa
import os
import glob

In [6]:
from DHS_To_Database.lib03_Update_Metadata import SurveyMetadataHelper

In [4]:
SPEC_SCHEMA = 'dhs_survey_specs'
DATA_SCHEMA = 'dhs_data_tables'

TABLESPEC_TABLENAME = 'dhs_table_specs_flat'
VALUESPEC_TABLENAME = 'dhs_value_descs'
SURVEYLIST_TABLENAME = 'dhs_survey_listing'

TABLE_SPEC_TABLE = ".".join((SPEC_SCHEMA, TABLESPEC_TABLENAME))
VALUE_SPEC_TABLE = ".".join((SPEC_SCHEMA, VALUESPEC_TABLENAME))
SURVEYLIST_TABLE = ".".join((SPEC_SCHEMA, SURVEYLIST_TABLENAME))

STAGING_FOLDER = "/mnt/c/Users/harry/OneDrive - Nexus365/Informal_Cities/DHS_Data_And_Prep/Staging"

In [5]:
engine = create_engine(conn_str_sqlalchemy)

# Part 1 - refresh DB cache of available surveys

As a separate, preliminary step, ensure that the cache we store of the available survey listing from the DHS API is up to date.

Load the list of all surveys from the DHS API. Note that in the past, this has been paginated, necessitating multiple calls to load it all. At present this isn't the case. As at 2021-01-08 there are 444 rows returned, so if you note less than this then it's likely pagination has been re-implemented.

In [6]:
api_survey_listing = pd.read_csv('http://api.dhsprogram.com/rest/dhs/surveys?f=csv&surveyStatus=all&perpage=2000')
api_survey_listing.columns = api_survey_listing.columns.str.lower()
api_survey_listing.head()

,surveyid,dhs_countrycode,surveyyear,surveytype,surveystatus,countryname,surveyyearlabel,surveynum,indicatordata,regionname,...,numberofhouseholds,universeofwomen,numberofwomen,minagewomen,maxagewomen,universeofmen,numberofmen,minagemen,maxagemen,numberoffacilities
0,AF2010OTH,AF,2010,OTH,Completed,Afghanistan,2010,348,0,South & Southeast Asia,...,22351.0,Ever Married Women,47848.0,12.0,49.0,NaN,NaN,NaN,NaN,NaN
1,AF2015DHS,AF,2015,DHS,Completed,Afghanistan,2015,471,1,South & Southeast Asia,...,24395.0,Ever Married Women,29461.0,15.0,49.0,Ever Married Men,10760.0,15.0,49.0,NaN
2,AF2018SPA,AF,2018,SPA,Completed,Afghanistan,2018-19,543,0,South & Southeast Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142.0
3,AF2021DHS,AF,2021,DHS,Ongoing,Afghanistan,2021,568,0,South & Southeast Asia,...,33120.0,Ever Married Women,37146.0,15.0,49.0,Ever Married Men,13672.0,15.0,49.0,NaN
4,AL2008DHS,AL,2008,DHS,Completed,Albania,2008-09,327,1,North Africa/West Asia/Europe,...,7999.0,All Women,7584.0,15.0,49.0,All Men,3013.0,15.0,49.0,NaN


In [7]:
our_survey_listing = pd.read_sql(f'SELECT * from {SURVEYLIST_TABLE}', con=engine)

See if there's any in the API results that aren't in our DB copy of the table

In [26]:
api_survey_listing[~api_survey_listing['surveynum'].isin(our_survey_listing['surveynum'])]


,surveyid,dhs_countrycode,surveyyear,surveytype,surveystatus,countryname,surveyyearlabel,surveynum,indicatordata,regionname,...,numberofhouseholds,universeofwomen,numberofwomen,minagewomen,maxagewomen,universeofmen,numberofmen,minagemen,maxagemen,numberoffacilities


If there's more in the API than our copy of the table, just drop and reload the whole table.

In [23]:
if len(api_survey_listing) > len(our_survey_listing):
    api_survey_listing.to_sql(name=SURVEYLIST_TABLENAME, con=engine, schema=SPEC_SCHEMA, 
                     index=False, if_exists='replace', method='multi')

# Part 2 - Update the database survey metadata tables

The database has one metadata table which records all the survey data tables and what columns they have (per survey). This contains the values from the parsed "FlatRecordSpec" csv files.

Another metadata table records all the values that all the survey data table columns can take, and (in the case of coded values) what the numeric values mean. This contains the values from the parsed "FlatValuesSpec" files.

For every CSV FlatRecordSpec and FlatValuesSpec file, we need to check if it's in the DB table at all (and load if not). If it is already there, then check if the info in the latest CSV matches what's in the DB. (As e.g. a re-released survey may change the specified width of some columns, or add some new data tables equating to extra metadata rows).

In [7]:
db_helper = SurveyMetadataHelper(conn_str=conn_str_sqlalchemy, table_spec_table=TABLESPEC_TABLENAME,
                             value_spec_table=VALUESPEC_TABLENAME, spec_schema=SPEC_SCHEMA)

Get a simple list of all the surveys that appear in the database metadata tables at all

In [8]:
existing_table_svys = db_helper.get_existing_table_surveys()
existing_value_svys = db_helper.get_existing_value_surveys()

In [9]:
existing_table_svys['surveyid'].values[:10]

array(['1', '100', '101', '105', '106', '109', '11', '110', '111', '112'],
      dtype=object)

Get a listing of all the up-to-date CSV table and value specification files we've parsed

In [10]:
spec_csv_dir = os.path.join(STAGING_FOLDER, 'parsed_specs')

tbl_files = glob.glob(os.path.join(spec_csv_dir, '*.FlatRecordSpec.csv'))
val_files = glob.glob(os.path.join(spec_csv_dir, '*.FlatValuesSpec.csv'))


In [ ]:
tbl_files[:5]

Each survey may have more than one file. This is normal if one is "individual recode" and another is "men's recode", but if there's some other reason then we need to investigate.

Create a list of all the files for each (numerically-identified) survey id

In [14]:
tbl_files_dict = SurveyMetadataHelper.build_spec_files_dict(tbl_files)
val_files_dict = SurveyMetadataHelper.build_spec_files_dict(val_files)

## Check that the input files are as expected

Run a few checks on the filenames of the parsed table spec files, to make sure they are for known data types (IR or MR), and whether there are multiple versions present for any surveys.

In [24]:
multiple_locs = set()
multiple_versions_per_filetype = set() # TODO
unknown_filetypes = set()
multiple_files_per_filetype = set()

for surveyid, dict_type_filenames in tbl_files_dict.items():
    survey_loc = None
    survey_version = {'ir':None, 'mr':None}
    types_this_survey = set()
    for filetype, fn_ver_locs in dict_type_filenames.items():
        
        # check we don't have anything except for IR and MR datasets
        if filetype not in ['ir', 'mr']:
            unknown_filetypes.add(surveyid)
        
        # check we don't have more than one IR or MR per survey
        # if we do, then it's a new version
        if len(fn_ver_locs)>1:
            multiple_files_per_filetype.add(surveyid)
        for fn, new_version, new_loc in fn_ver_locs:
            # check for location being unique across IR and MR files
            if survey_loc is None:
                survey_loc = new_loc
            elif survey_loc != new_loc:
                multiple_locs.add(surveyid)
        
            # version string is often different between IR and MR files, that's ok, 
            # we want to check for multiple versions per filetype
            if survey_version[filetype] is None:
                survey_version[filetype] = new_version
            elif survey_version[filetype] != new_version:
                multiple_versions_per_filetype.add(surveyid)

        

**Check if there are any surveys with filetypes other than IR/MR?**

In [25]:
unknown_filetypes

set()

**Check for any unexpected files**

We expect that we may have both IR and MR files for a survey i.e. up to two tablespec files. If we have more than two files per (numeric) survey then it's either due to there being different location identifiers (as with survey 156 where they released a national dataset plus multiple regional ones); or to some different filetype being present other than IR/MR; or to there being multiple versions present of the same survey (e.g. if we've re-downloaded and are working in the same folder after an update was released). Check which surveys are affected by this. 

This shows all surveys with more than one file per filetype, could be due to multiple locations OR versions:

In [26]:
multiple_files_per_filetype

{'580'}

This shows those truly with multiple versions per filetype at same location. If so then we will drop and reload the latest one only. (It's also quite common for the IR and MR files to have a different version. Don't worry about this.)

In [27]:
multiple_versions_per_filetype

{'580'}

In the case of one survey, number 156 (India) the download contains a main data file for India nationally (IAIR42), plus one for each state. (These break the convention by which the first two characters of the filename give the country code of the survey, because they are named for the state instead e.g. the file KEIR42 does not refer to Kenya but Kerala. This is one reason why we prepend and use the numeric survey id instead.) We do not need these state-wise files. If there's any like this, just move them out of the folder and re-run.

Here we check that there are no other surveys affected by this other than 156:

In [28]:
multiple_locs

set()

we only want to parse and load one version per filetype (IR/MR) and per survey.

### Check and load/reload column spec files as necessary

Although we've made various checks already, we'll still re-check in the processing loop that there aren't any unexpected files, and we'll also check that the database is consistent for each survey e.g. that it does not already contain multiple versions.


In [17]:
db_helper._is_dry_run = False

In [ ]:
good_tbl_svys = []
drop_tbl_svys = []
new_tbl_svys = []

good_val_svys = []
reload_val_svys = []
new_val_svys = []
               
def report_cols_all_good(surveyid, file_type, file_name):
    print(f"Survey {surveyid} {file_type.upper()} spec file ({file_name}) is already completely loaded")
    good_tbl_svys.append(os.path.basename(file_name).split('.')[0])
    
for surveyid, dict_type_filenames in tbl_files_dict.items():
    some_only_in_csv = False
    some_only_in_db = False
    
    locset = set()
    has_dupes = False
    for file_type, fn_ver_locs in dict_type_filenames.items():
        for _, _, loc in fn_ver_locs:
            locset.add(loc)
        if len(locset)>1:
            warnings.warn(f"""
            ****** WARNING MULTIPLE LOCATIONS FOUND FOR SURVEYID {surveyid} FILETYPE {file_type}. CANNOT DETERMINE WHICH TO LOAD *****
            ****** PLEASE REMOVE SURPLUS FILES AND RERUN FOR THIS WHOLE SURVEY*****
            """)
            has_dupes = True
    if has_dupes:
        continue
    # shortcut other checks, if nothing for this survey is in DB at all then we definitely need to load
    need_to_load_all_data = surveyid not in existing_table_svys.values
    if need_to_load_all_data: # no files or part files from this svy loaded
        for file_type, fn_ver_locs in dict_type_filenames.items():
            # the versions are strings such as '70' or '7A' where the first character 
            # represents the phase (fixed for a survey) and the second the actual version 
            # of this file. Anyway, natural sort order by version is ok.
            latest = sorted(fn_ver_locs, key=lambda tup: tup[1])[-1]
            file_name, file_ver, file_loc = latest
            print(f"No data for survey {surveyid} / {file_type} are present: "+
                  f"loading latest data for {file_type.upper()} from {file_name}")
            if len(fn_ver_locs) > 1: print(f"   ({len(fn_ver_locs)} versions were available, loading {file_ver}")
            db_helper.load_new_metadata_file(file_name)
        continue
    
    # metadata for this survey are already present. That doesn't necessarily mean we don't need to 
    # load or reload this file. Check.
    for file_type, fn_ver_locs in dict_type_filenames.items():
        latest = sorted(fn_ver_locs, key=lambda tup: tup[1])[-1]
        file_name, file_ver, file_loc = latest
        
        # Check for the scenario where IR data are in the DB but MR data aren't, etc
        if not db_helper.get_any_in_db_cols(surveyid, file_type):
            print(f"{file_type.upper()} data for survey {surveyid} are not present: loading from {file_name}")
            if len(fn_ver_locs) > 1: print(f"    (of {len(fn_ver_locs)} available versions)")
            db_helper.load_new_metadata_file(file_name)
            continue
            
        db_ver, db_has_single_ver = db_helper.get_db_survey_version_cols(surveyid, file_type)
        if not db_has_single_ver:
            db_helper.drop_and_reload(file_name, 
                                      msg="Multiple versions found in DB, dropping all and loading latest")
        
        elif file_ver > db_ver: # version is alphanumeric but lexical comparison works
            # this needs to be done for the data tables too
            db_helper.drop_and_reload(file_name, 
                                      msg=f"Newer version found - DB is version {db_ver} "+
                                      f"and file is version {file_ver}")
        else:
            # check that the same rows are present in the CSV and DB
            file_cols_data = pd.read_csv(file_name)
            db_cols_data = db_helper.get_tablespec_rows_for_svy_filetype(surveyid, file_type)
            # do an outer join to find items present in files but not DB and vice versa
            merged_cols = file_cols_data.merge(db_cols_data, how='outer',
                                      left_on=['Name','RecordName','Label','Len'],
                                      right_on=['name','recordname', 'label', 'len'],
                                      suffixes=('_new', '_exist'),
                                      indicator=True)
            cols_all_good = len(merged_cols[merged_cols['_merge']!='both']) == 0
            if not cols_all_good:
                # Most likely reason for having new columns info is that we've added men's recode data,
                # but that won't be the case here as the check included filetype.
                # Reasons to be here would be it's a re-issued survey (same id number) and has added 
                # different columns (in which case there will be some left-only and no right-only ones), 
                # and/or that some column labels have changed (e.g. prefix of NA been added).
                some_only_in_db = len(merged_cols[merged_cols['_merge'] == 'right_only']) > 0
                some_only_in_csv = len(merged_cols[merged_cols['_merge'] == 'left_only']) > 0
                if some_only_in_csv and some_only_in_db:
                    # Not worth the effort to investigate: just drop everything from this survey and reload.
                    db_helper.drop_and_reload(file_name, 
                                              msg="Rows mismatch between file and DB")
                elif some_only_in_csv:
                    # Not worth the effort to investigate: just drop everything from this survey and reload.
                    db_helper.drop_and_reload(file_name, 
                                              msg="Rows present in file but not in DB")
                elif some_only_in_db:
                    # Not worth the effort to investigate: just drop everything from this survey and reload.
                    db_helper.drop_and_reload(file_name, 
                                              msg="Rows present in DB but not in file")
            else:
                report_cols_all_good(surveyid, file_type, file_name)
        

### Check and load/reload value spec files as necessary

In [ ]:
good_tbl_svys = []
drop_tbl_svys = []
new_tbl_svys = []

good_val_svys = []
reload_val_svys = []
new_val_svys = []


def report_vals_all_good(surveyid, file_type, file_name):
    print(f"Survey {surveyid} {file_type.upper()} values file ({file_name}) is already completely loaded")
    good_val_svys.append(os.path.basename(file_name).split('.')[0])
    
for surveyid, dict_type_filenames in val_files_dict.items():
    #if not surveyid.startswith('1'):
    #    continue
    some_only_in_csv = False
    some_only_in_db = False
    #if surveyid != '16':
     #   continue
    
    locset = set()
    has_dupes = False
    for file_type, fn_ver_locs in dict_type_filenames.items():
        for _, _, loc in fn_ver_locs:
            locset.add(loc)
        if len(locset)>1:
            print(f"""
            ****** WARNING MULTIPLE LOCATIONS FOUND FOR SURVEYID {surveyid} FILETYPE {file_type}. CANNOT DETERMINE WHICH TO LOAD *****
            ****** PLEASE REMOVE SURPLUS FILES AND RERUN FOR THIS WHOLE SURVEY*****
            """)
            has_dupes = True
    if has_dupes:
        continue
        
    # shortcut other checks, if nothing for this survey is in DB at all then we definitely need to load
    need_to_load_all_data = surveyid not in existing_value_svys.values
    if need_to_load_all_data: # no files or part files from this svy loaded
        for file_type, fn_ver_locs in dict_type_filenames.items():
            # the versions are strings such as '70' or '7A' where the first character 
            # represents the phase (fixed for a survey) and the second the actual version 
            # of this file. Anyway, natural sort order by version is ok.
            latest = sorted(fn_ver_locs, key=lambda tup: tup[1])[-1]
            file_name, file_ver, file_loc = latest
            print(f"No data for survey {surveyid} / {file_type} are present: "+
                  f"loading latest data for {file_type.upper()} from {file_name}")
            if len(fn_ver_locs) > 1: print(f"   ({len(fn_ver_locs)} versions were available, loading {file_ver}")
            db_helper.load_new_metadata_file(file_name)
        continue
    for file_type, fn_ver_locs in dict_type_filenames.items():
        latest = sorted(fn_ver_locs, key=lambda tup: tup[1])[-1]
        file_name, file_ver, file_loc = latest
        
        # Check for the scenario where IR data are in the DB but MR data aren't, etc
        if not db_helper.get_any_in_db_vals(surveyid, file_type):
            print(f"{file_type.upper()} value-spec data for survey {surveyid} are not present: loading from {file_name}")
            if len(fn_ver_locs) > 1: print(f"    (of {len(fn_ver_locs)} available versions)")
            db_helper.load_new_metadata_file(file_name)
            continue
            
        db_ver, db_has_single_ver = db_helper.get_db_survey_version_vals(surveyid, file_type)
        if not db_has_single_ver:
            db_helper.drop_and_reload(file_name,
                            msg="Multiple versions found in DB, dropping all and loading latest")
        elif file_ver > db_ver:
            # this needs to be done for the data tables too
            db_helper.drop_and_reload(file_name,
                            msg=f"Newer version found - DB is version {db_ver} "+
                                      "and file is version {file_ver}")
        else:
            # check that the same rows are present in the CSV and DB
            file_vals_data = pd.read_csv(file_name)
            db_vals_data = db_helper.get_valuespec_rows_for_svy_filetype(surveyid, file_type)
            # do an outer join to find items present in files but not DB and vice versa
            merged_vals = file_vals_data.merge(db_vals_data, how='outer',
                                      left_on=['Name','Value','ValueDesc','ValueType'],
                                      right_on=['col_name','value', 'value_desc', 'value_type'],
                                      suffixes=('_new', '_exist'),
                                      indicator=True)
            vals_all_good = len(merged_vals[merged_vals['_merge']!='both']) == 0
            if not vals_all_good:
                # Most likely reason is that we now have men's recode data as well, but did not previously, 
                # but we have checked for that already.
                # Other reasons would be it's a re-issued survey (same id number) and has added different columns 
                # (in which case there will be some left-only and no right-only ones), and/or that some column 
                # labels have changed (e.g. prefix of NA been added).
                some_only_in_db = len(merged_vals[merged_vals['_merge'] == 'right_only']) > 0
                some_only_in_csv = len(merged_vals[merged_vals['_merge'] == 'left_only']) > 0
                if some_only_in_csv and some_only_in_db:
                    # Not worth the effort to investigate: just drop everything from this survey and reload.
                    db_helper.drop_and_reload(file_name, 
                                              msg="Rows mismatch between file and DB")
                elif some_only_in_csv:
                    # Not worth the effort to investigate: just drop everything from this survey and reload.
                    db_helper.drop_and_reload(file_name, 
                                              msg="Rows present in file but not in DB")
                elif some_only_in_db:
                    # Not worth the effort to investigate: just drop everything from this survey and reload.
                    db_helper.drop_and_reload(file_name, 
                                              msg="Rows present in DB but not in file")
                    
            else:
                #pass
                # TODO check 475
                report_vals_all_good(surveyid, file_type, file_name)
        

### Correcting errors

Before this code was implemented, a survey occasionally got loaded twice to the column/value descriptions tables .

#### Value descriptions

Find duplicate rows in the value descriptions table, according to the surveyid, column name and value
```sql 
SELECT count(*) as n_occ, surveyid, col_name, value 
FROM dhs_survey_specs.dhs_value_descs 
GROUP BY surveyid, col_name, value
HAVING count(*)>1
```

In [14]:
duplicate_values = pd.read_sql('''SELECT count(*) as n_occ, surveyid, col_name, value 
FROM dhs_survey_specs.dhs_value_descs 
GROUP BY surveyid, col_name, value
HAVING count(*)>1''', con=engine)
duplicate_values.head()

There seem to quite often be a few duplicates in weird calendar fields that I have not properly looked into. 

Consider there to be a problem if there are more than 1000 duplicates per survey

In [17]:
pd.options.display.max_rows=200
n_dupes = duplicate_values.groupby(['surveyid']).size().reset_index(name='counts')
dodgy_vals = n_dupes[n_dupes['counts']>1000]
repeat_val_load_str = ",".join(["'"+str(s)+"'" for s in dodgy_vals['surveyid']])
repeat_val_load_str

''

In [24]:
delete_str = f"DELETE FROM dhs_survey_specs.dhs_value_descs WHERE surveyid in ({repeat_val_load_str});"
print(delete_str)

DELETE FROM dhs_survey_specs.dhs_value_descs WHERE surveyid in ('5','48','49','50','51','52','53','473','474','475','476','477','478','481','483','484','485','487','489','490','491','493','497','499','500','503','504','505','509','510','511','512','514','515','516','517','521','522','523','526','527','528','529','534');


Execute this to delete the misloaded data, then reload them using the code in this notebook

#### Column descriptions

In [19]:
duplicate_cols = pd.read_sql('''SELECT count(*) as n_occ, surveyid, recordname, name
FROM dhs_survey_specs.dhs_table_specs_flat 
GROUP BY surveyid, recordname, name
HAVING count(*)>1''', con=engine)
duplicate_cols

,n_occ,surveyid,recordname,name


In [20]:
duplicate_cols['surveyid'].unique()

pd.options.display.max_rows=200
n_dupe_cols = duplicate_cols.groupby(['surveyid']).size().reset_index(name='counts')

In [21]:
n_dupe_cols

,surveyid,counts


In [22]:
dodgy_cols = n_dupe_cols[n_dupe_cols['counts']>50]
repeat_col_load_str = ",".join(["'"+str(s)+"'" for s in dodgy_cols['surveyid']])
dodgy_col_surveys=list(dodgy_cols['surveyid'])
repeat_col_load_str

''

In [23]:
delete_str = f"DELETE FROM dhs_survey_specs.dhs_table_specs_flat WHERE surveyid in ({repeat_col_load_str});"
print(delete_str)

DELETE FROM dhs_survey_specs.dhs_table_specs_flat WHERE surveyid in ();
